In [17]:
import numpy as np
import pandas as pd


def _psi(expected: np.ndarray, actual: np.ndarray, bucket_type: str = "bins", n_bins: int = 10) -> float:
    """Calculate PSI metric for two arrays.

    Parameters
    ----------
        expected : list-like
            Array of expected values
        actual : list-like
            Array of actual values
        bucket_type : str
            Binning strategy. Accepts two options: 'bins' and 'quantiles'. Defaults to 'bins'.
            'bins': input arrays are splitted into bins with equal
                and fixed steps based on 'expected' array
            'quantiles': input arrays are binned according to 'expected' array
                with given number of n_bins
        n_bins : int
            Number of buckets for binning. Defaults to 10.

    Returns
    -------
        A single float number
    """
    breakpoints = np.arange(0, n_bins + 1) / (n_bins) * 100
    if bucket_type == "bins":
        breakpoints = np.histogram(expected, n_bins)[1]
    elif bucket_type == "quantiles":
        breakpoints = np.percentile(expected, breakpoints)

    # Calculate frequencies
    expected_percents = np.histogram(expected, breakpoints)[0] / len(expected)
    actual_percents = np.histogram(actual, breakpoints)[0] / len(actual)
    # Clip freaquencies to avoid zero division
    expected_percents = np.clip(expected_percents, a_min=0.0001, a_max=None)
    actual_percents = np.clip(actual_percents, a_min=0.0001, a_max=None)
    # Calculate PSI
    psi_value = (expected_percents - actual_percents) * np.log(expected_percents / actual_percents)
    psi_value = sum(psi_value)

    return psi_value


def calculate_psi(
        expected: np.ndarray, actual: np.ndarray, bucket_type: str = "bins", n_bins: int = 10, axis: int = 0
) -> np.ndarray:
    """Apply PSI calculation to 2 1-d or 2-d arrays.

    Parameters
    ----------
    expected : list-like
        Array of expected values
    actual : list-like
        Array of actual values
    bucket_type : str
        Binning strategy. Accepts two options: 'bins' and 'quantiles'. Defaults to 'bins'.
            'bins' - input arrays are splitted into bins with equal
                and fixed steps based on ’expected' array
            'quantiles' - input arrays are binned according to ’expected’ array
                with given number of n_bins
    n_bins : int
        Number of buckets for binning. Defaults to 10.

    Returns
    -------
        np.ndarray
    """
    if len(expected.shape) == 1:
        psi_values = np.empty(len(expected.shape))
    else:
        psi_values = np.empty(expected.shape[axis])

    for i in range(0, len(psi_values)):
        if len(psi_values) == 1:
            psi_values = _psi(expected, actual, bucket_type, n_bins)
        elif axis == 0:
            psi_values[i] = _psi(expected[:, i], actual[:, i], bucket_type, n_bins)
        elif axis == 1:
            psi_values[i] = _psi(expected[i, :], actual[i, :], bucket_type, n_bins)
        return np.array(psi_values)

In [18]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data-20130606-structure-20141127.csv')

In [19]:
df

,Год,Отчетный период,Значение,Единица измерения,Код разреза,Код признака
0,2010,значение показателя за год,103335.95,Тысяча рублей,mОКАТО,643
1,2010,значение показателя за год,1054.75,Тысяча рублей,mОКАТО,30
2,2010,значение показателя за год,0.00,Тысяча рублей,mОКАТО,140000000001
3,2010,значение показателя за год,0.00,Тысяча рублей,mОКАТО,150000000001
4,2010,значение показателя за год,0.00,Тысяча рублей,mОКАТО,170000000001
...,...,...,...,...,...,...
179,2011,значение показателя за год,0.00,Тысяча рублей,mОКАТО,100000000001
180,2011,значение показателя за год,0.00,Тысяча рублей,mОКАТО,440000000001
181,2011,значение показателя за год,129.50,Тысяча рублей,mОКАТО,640000000001
182,2011,значение показателя за год,0.00,Тысяча рублей,mОКАТО,990000000001


In [20]:
df_excpected = df[df['Год']==2010]['Значение']

In [21]:
df_excpected.describe()

,Значение
count,92.000000
mean,3369.650652
std,17031.816374
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,103335.950000


In [22]:
df_actual = df[df['Год']==2011]['Значение']

In [23]:
df_actual.describe()

,Значение
count,92.000000
mean,143.466292
std,674.197695
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,4399.632950


In [24]:
calculate_psi(df_excpected, df_actual, bucket_type='bins', n_bins=10, axis=0)

array(0.16803)

In [25]:
calculate_psi(df_excpected, df_excpected, bucket_type='bins', n_bins=10, axis=0)

array(0.)

In [26]:
df_actual = np.where(df['Значение'] > 1000000, 0, df['Значение'])

In [27]:
calculate_psi(df_excpected, df_actual, bucket_type='bins', n_bins=10, axis=0)

array(0.01157382)